In [3]:
%load_ext autoreload
%autoreload 2
%matplotlib inline  

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import numpy as np
from os.path import dirname, join as pjoin
import scipy.io as sio
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import TensorDataset, DataLoader, random_split

from func2graph import data, baselines

In [21]:
# sessions are preprocessed by sliding windows, num_unqiue_cell_types and num_unique_neurons are known
session1 = torch.rand(32, 10, 100)
session2 = torch.rand(30, 13, 100)
session1_cell_type_ids = torch.rand(32, 10)
session2_cell_type_ids = torch.rand(30, 13)
session1_neuron_ids = torch.rand(32, 10)
session2_neuron_ids = torch.rand(30, 13)

all_sessions = [session1, session2]
all_sessions_cell_type_ids = [session1_cell_type_ids, session2_cell_type_ids]
all_sessions_neuron_ids = [session1_neuron_ids, session2_neuron_ids]

class Mouse_Session_Dataset(TensorDataset):
    def __init__(
        self, 
        all_sessions, 
        all_sessions_cell_type_ids, 
        all_sessions_neuron_ids,
        batch_size=3,                      # real batch size
    ):
        num_batch_per_session = [session.shape[0] // batch_size for session in all_sessions]

        self.all_batch = []
        self.all_batch_cell_type_ids = []
        self.all_batch_neuron_ids = []
        for i in range(len(num_batch_per_session)):      # for each session
            for j in range(num_batch_per_session[i]):      # for each batch
                self.all_batch.append(all_sessions[i][j*batch_size:(j+1)*batch_size])
                self.all_batch_cell_type_ids.append(all_sessions_cell_type_ids[i][j*batch_size:(j+1)*batch_size])
                self.all_batch_neuron_ids.append(all_sessions_neuron_ids[i][j*batch_size:(j+1)*batch_size])

    def __getitem__(self, index):
        return self.all_batch[index], self.all_batch_cell_type_ids[index], self.all_batch_neuron_ids[index]

    def __len__(self):
        return len(self.all_batch)



dataset = Session_Dataset(all_sessions, all_sessions_cell_type_ids, all_sessions_neuron_ids)
dataloader = DataLoader(dataset, batch_size=1, shuffle=False)    # this is not real batch_size
for idx, (batch, batch_cell_type_ids, batch_neuron_ids) in enumerate(dataloader):
    batch = batch.squeeze(0)                 # remove the fake batch_size
    batch_cell_type_ids = batch_cell_type_ids.squeeze(0)
    batch_neuron_ids = batch_neuron_ids.squeeze(0)
    print(idx, batch.shape, batch_cell_type_ids.shape, batch_neuron_ids.shape)

    if idx == 11:
        print(batch_cell_type_ids[1])
        print(batch_neuron_ids[1])
        print(session2_cell_type_ids[4])
        print(session2_neuron_ids[4])

0 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
1 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
2 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
3 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
4 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
5 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
6 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
7 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
8 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
9 torch.Size([3, 10, 100]) torch.Size([3, 10]) torch.Size([3, 10])
10 torch.Size([3, 13, 100]) torch.Size([3, 13]) torch.Size([3, 13])
11 torch.Size([3, 13, 100]) torch.Size([3, 13]) torch.Size([3, 13])
tensor([0.4689, 0.9265, 0.4217, 0.1391, 0.4735, 0.7944, 0.0489, 0.8042, 0.6423,
        0.4617, 0.3809, 0.0762, 0.6617])
tensor([0.7390, 0.3785, 0.2016, 0.1473, 0.6801, 0.2504, 0.8860, 0.9147, 0

In [35]:
# preprocess sessions by sliding windows
# determine num_unqiue_cell_types and num_unique_neurons
# determine unique_cell_type_ids and unique_neuron_ids

# a list of sessions files
input_sessions_files = []
output_folder = '../../data/Mouse/preprocessed/'

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-07/'
input_setting = 'Blank/01/'
session_normalization = 'session'

accumulate_UnqiueID = []

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-04/'
input_setting = 'Blank/01/'
session_normalization = 'session'

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-08/'
input_setting = 'Blank/01/'
session_normalization = 'session'

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-09/'
input_setting = 'Blank/01/'
session_normalization = 'session'

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-23/'
input_setting = 'Blank/01/'
session_normalization = 'session'

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

directory = '../../data/Mouse/Bugeon/'
date_exp = 'SB025/2019-10-24/'
input_setting = 'Blank/01/'
session_normalization = 'session'

activity, frame_times, UniqueID, neuron_ttypes = data.load_mouse_data_session(
    directory, date_exp, input_setting, session_normalization
)
accumulate_UnqiueID.append(UniqueID)

# print(activity.shape)
# print(UniqueID)
# print(neuron_ttypes)

# sort UniqueID
accumulate_UnqiueID = np.concatenate(accumulate_UnqiueID)
print(np.sort(accumulate_UnqiueID[~np.isnan(accumulate_UnqiueID)]))

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  34.  35.  35.  36.  37.  38.  39.
  40.  41.  41.  41.  42.  43.  44.  45.  45.  45.  45.  46.  47.  47.
  48.  48.  48.  48.  49.  50.  51.  52.  53.  53.  53.  53.  54.  55.
  56.  57.  58.  58.  59.  59.  60.  60.  61.  61.  62.  63.  64.  65.
  65.  65.  66.  67.  68.  69.  70.  71.  72.  72.  73.  74.  75.  75.
  76.  77.  78.  79.  79.  80.  81.  82.  82.  83.  83.  84.  85.  85.
  85.  85.  86.  87.  87.  88.  88.  89.  90.  91.  91.  92.  92.  93.
  93.  94.  95.  96.  96.  96.  97.  98.  98.  98.  98.  99. 100. 100.
 101. 101. 101. 102. 102. 102. 103. 103. 103. 104. 105. 106. 107. 108.
 108. 108. 109. 110. 110. 111. 111. 112. 113. 114. 115. 116. 117. 117.
 117. 117. 118. 118. 118. 119. 120. 120. 121. 122. 123. 124. 125. 126.
 127. 127. 128. 129. 130. 131. 131. 132. 132. 133. 133. 133. 134. 134.
 135. 

In [48]:
def assign_unique_neuron_ids(all_sessions_original_UniqueID, num_neurons_per_session):
    """
    all_sessions_original_UniqueID: a list of UniqueID from all sessions
    num_neurons_per_session: a list of number of neurons from all sessions
    """

    # first reassign ID starting from 0 to those non-NaN neurons
    # same IDs should be assigned to neurons that have the same UniqueID
    non_nan_values = all_sessions_original_UniqueID[~np.isnan(all_sessions_original_UniqueID)]

    unique_non_nan_values = np.unique(non_nan_values)
    print(unique_non_nan_values)
    print(len(unique_non_nan_values))
    id_mapping = {unique_non_nan_values[i]: i for i in range(len(unique_non_nan_values))}

    new_ids = [id_mapping[non_nan_values[i]] for i in range(len(non_nan_values))]
    all_sessions_new_UniqueID = np.copy(all_sessions_original_UniqueID)
    all_sessions_new_UniqueID[~np.isnan(all_sessions_new_UniqueID)] = new_ids

    print(all_sessions_original_UniqueID[:100])
    print(all_sessions_new_UniqueID[:100])

    # then assign new IDs to those NaN neurons
    num_non_nan = np.sum(~np.isnan(all_sessions_original_UniqueID))     # new IDs start from num_non_nan
    num_nan = np.sum(np.isnan(all_sessions_original_UniqueID))           # new IDs end with num_non_nan + num_nan -1

    new_ids = np.arange(num_non_nan, num_non_nan + num_nan)
    all_sessions_new_UniqueID[np.isnan(all_sessions_new_UniqueID)] = new_ids

    print(all_sessions_new_UniqueID[:100])

    # check if the number of unique ids is the same as the length of array
    unique_ids = np.unique(all_sessions_new_UniqueID)
    print(len(unique_ids))
    print(len(all_sessions_new_UniqueID))


assign_unique_neuron_ids(UniqueID, [len(UniqueID)])

[  0.   1.   2.   3.   4.   5.   7.  22.  24.  26.  27.  38.  45.  48.
  53.  57.  61.  62.  69.  70.  71.  78.  85.  87.  88.  95.  98. 101.
 102. 104. 111. 112. 117. 118. 123. 133. 139. 149. 153. 154. 156. 157.
 158. 159. 160. 162. 166. 168. 169. 170. 172. 177. 178. 179. 184. 187.
 188. 196. 198. 199. 201. 205. 207. 208. 212. 213. 216. 217. 218. 220.
 223. 237. 240. 241. 242. 248. 252. 260. 261. 265. 269. 273. 275. 276.
 277. 278. 279. 280. 281. 285. 287. 288. 289. 296. 298. 299. 300. 301.
 304. 305. 306. 312. 314. 318. 326. 327. 334. 339. 346. 352. 356. 357.
 358. 361. 363.]
115
[ nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan 188.  nan 198.
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan  nan
  nan  nan  nan  nan  nan 169.  nan 240. 241. 260.  nan 